# Introduction to metaprogramming

Normal programs manipulate data. Metaprograms **manipulate code**. (In many languages this is impossible.)
Of course, the ultimate goal is that the programs that we manipulate will later operate on data.

Julia has strong **metaprogramming** capabilities. What does this mean?

> **meta**: something on a higher level

**metaprogramming** = "programming a program"

i.e. writing code (a program) to manipulate not data, but code. (And this code will, itself, then manipulate data [or another program...].)

In [ ]:
Conjecture:

## Expressions

We use the special syntax `:( ... )` to represent a piece of Julia code:

In [54]:
ex = :(x+3)

:(x + 3)

We can look at the structure of the object `ex` with the `dump` function:

In [55]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Int64 3


In [56]:
typeof(ex)

Expr

We see that `ex` is of type `Expr`, meaning a Julia expression object - a piece of Julia code, stored as an object inside Julia itself!

The details about the object are stored (as usual) in its fields. The kind of expression object is stored as the `head` field:

In [57]:
ex.head

:call

In this case, we see that it represents a function call. 

The arguments of the function call are stored in the `ex.args` field:

In [58]:
ex.args

3-element Array{Any,1}:
  :+
  :x
 3  

In [59]:
ex.args[2]

:x

We can now *manipulate* and *modify* the object by *changing* these fields:

In [60]:
ex.args[2] = :z
ex.args[3] = :4

4

In [61]:
ex

:(z + 4)

We see that the expression has changed. We can even change it from an addition to something else:

In [62]:
ex.args[1] = :*

:*

In [63]:
ex

:(z * 4)

This is the basis of metaprogramming, since it gives us the ability to take one piece of code, and modify it to produce another piece of code.

The foundation of an expression are `Symbol`s, written with initial colons (`:`):


In [64]:
typeof(:z)

Symbol

A more compact form to display an expression is as follows:

In [65]:
using MacroTools

In [66]:
Meta.show_sexpr(ex)

(:call, :*, :z, 4)

## More complicated expressions

What about more complicated expressions?

In [67]:
ex2 = :(x + 3y)

:(x + 3y)

In [68]:
Meta.show_sexpr(ex2)

(:call, :+, :x, (:call, :*, 3, :y))

In [69]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 3
        3: Symbol y


In [70]:
ex2.args[3]

:(3y)

In [71]:
typeof(ex2.args[3])

Expr

We see that expressions are recursive, in the sense that the subpiece `:(3y)` is itself another `Expr` object.

Julia provides a mechanism to create an `Expr` from a `String`, via the `parse` function:

In [74]:
Meta.parse("x + 3y")

:(x + 3y)

## Manipulating expressions

How can we do something more complicated? For example, suppose we start with the expression

In [75]:
ex = :(x + 3x^2)

:(x + 3 * x ^ 2)

and we would like to change *all* `x`s to `(x+1)`s? We will need to dig down into the structure.
A first attempt is the following function. Note that this *modifies* its argument, so has a `!` ("bang") in the function name.

In [76]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        
        if args[i] == :x
            args[i] = :(x+1)
        end
        
    end
end

traverse! (generic function with 1 method)

In [77]:
traverse!(ex)

In [78]:
ex

:((x + 1) + 3 * x ^ 2)

Oops, that didn't work -- we didn't manage to reach inside the inner `Expr` object. 

In [79]:
ex = :(x + 3x^2)

:(x + 3 * x ^ 2)

In [80]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        if args[i] == :x
            args[i] = :(x+1)
        end
    end
end

traverse! (generic function with 1 method)

In [81]:
traverse!(ex)

(i, args[i]) = (1, :+)
(i, args[i]) = (2, :x)
(i, args[i]) = (3, :(3 * x ^ 2))


The AST ("Abstract Syntax Tree") has a **recursive** structure, so we use a recursive function:

In [82]:
Meta.show_sexpr(ex)

(:call, :+, (:call, :+, :x, 1), (:call, :*, 3, (:call, :^, :x, 2)))

In [83]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        
        if isa(args[i], Expr)
            traverse!(args[i])
            
        elseif args[i] == :x
            args[i] = :(x+1)
        end
    end
end

traverse! (generic function with 1 method)

In [84]:
traverse!(ex)

(i, args[i]) = (1, :+)
(i, args[i]) = (2, :(x + 1))
(i, args[i]) = (1, :+)
(i, args[i]) = (2, :x)
(i, args[i]) = (3, 1)
(i, args[i]) = (3, :(3 * x ^ 2))
(i, args[i]) = (1, :*)
(i, args[i]) = (2, 3)
(i, args[i]) = (3, :(x ^ 2))
(i, args[i]) = (1, :^)
(i, args[i]) = (2, :x)
(i, args[i]) = (3, 2)


In [85]:
ex

:(((x + 1) + 1) + 3 * (x + 1) ^ 2)

**Exercise**: Make the `traverse!` function more general, to replace `x` by an arbitrary expression that is another argument of the function.

## Evaluation

Now that we have our new object, we want to use it in Julia. We do this using `eval`:


In [93]:
eval(x) = Core.eval(Main, x)

eval (generic function with 1 method)

In [94]:
ex = :(3x^2)

:(3 * x ^ 2)

In [95]:
eval(ex)

27

In [96]:
x = 3

3

In [97]:
eval(ex)

27

## Using multiple dispatch with expressions

What if, say, we want to wrap "number literals" like 3, 4.5 with `f(3)` etc.?
We will write a more Julian version, using **multiple dispatch**:

In [98]:
function wrap_literals(ex::Expr, f::Symbol)
    
    #println("Entering !")
    #@show ex, s, new_expr
    
    args = ex.args
    
    for i in 1:length(args)
        #@show i, args[i]
        args[i] = wrap_literals(args[i], f)
        
    end
    
    return ex
end

wrap_literals(x::Number, f::Symbol) = :($f($x))
wrap_literals(x, f) = x  # fall-back method

wrap_literals (generic function with 3 methods)

In [99]:
ex = :(3x^2)
wrap_literals(:(3x^2), :f)

:(f(3) * x ^ f(2))

## Inserting pieces of code

Wilkinson-type polynomials are polynomials like

$p_5(x) = (x-1) (x-2) (x-3) (x-4) (x-5)$

In Julia we could write

In [100]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

p_5 (generic function with 1 method)

**Exercise:** We can interpolate pieces of code in other pieces using the `$` operator, and symbols from strings using `Symbol`. Make and evaluate an expression to define $p_n$ for different $n$, and interact with it.

## Generating repetitive code

One of the uses of metaprogramming is **code generation**. 
Suppose that for some reason we want to make a type that behaves like a `Float64`, but with some additional functionality.:

In [102]:
struct MyFloat
    a::Float64
end

We would need to write methods like


In [103]:
import Base: +, -

+(x::MyFloat, y::MyFloat) = x.a + y.a
-(x::MyFloat, y::MyFloat) = x.a - y.a

- (generic function with 176 methods)

This will get boring fast. Whenever we have repetition like this, we should get the computer to do it for us. However, here we need to **repeat code**. Let's make a code *template*; it should look like

    op(x::MyFloat, y::MyFloat) = op(x.a, y.a)

We want a code object that looks like that:

In [106]:
ex = prettify(:(op(x::MyFloat, y::MyFloat) = op(x.a, y.a)))

:(op(x::MyFloat, y::MyFloat) = op(x.a, y.a))

We can also write this as 

In [108]:
ex = quote
        op(x::MyFloat, y::MyFloat) = op(x.a, y.a)
    end |> prettify

:(op(x::MyFloat, y::MyFloat) = op(x.a, y.a))

However, we want to replace `op` **by its value**

In [110]:
op = :+
ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end |> prettify

:(x::MyFloat + y::MyFloat = x.a + y.a)

And now just loop:

In [111]:
import Base: +,-,*,/

for op in (:+, :-, :*, :/)
    
    ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end
    
    eval(ex)
end


Julia provides `@eval` to make this shorter:

In [112]:
for op in (:+, :-, :*, :/)
    
    @eval $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    
end


## Macros

We don't want the user to have to think about Julia syntax objects. We can supply a **macro**.
This can be thought of as a "meta-function": it takes a Julia expression object as argument, and returns a Julia expression object. However, when it is called, it *automatically parses its input into the expression object*.

The simplest example:

In [113]:
macro simple(ex)
    show(ex)
    return nothing
end
    

@simple (macro with 1 method)

In [114]:
@simple 3x

:(3x)

We can now make a macro `wrap` that calls our function:

In [120]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    show(new_ex)
end

@wrap (macro with 1 method)

In [121]:
@wrap g 3x+2

:(g(3) * x + g(2))

We can also call it as `@wrap(g, 3x+2)`:

In [122]:
@wrap(g, 3x+2)

:(g(3) * x + g(2))

In fact, macros should **return an expression**. This expression will be **evaluated in the scope in which it is called**:

In [123]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    return new_ex
end

@wrap (macro with 1 method)

In [126]:
@wrap(g, 3x+2)

UndefVarError: UndefVarError: g not defined

In [127]:
x = 3
g(x) = 10x

g (generic function with 1 method)

In [128]:
@wrap g 3x+2

110

In [129]:
g(3)*x+g(2)

110